# Language & Automatas. Unit III. Assignment 06.
---
**Data 7A**

* Fatima Martínez Torres
* Mario Alberto Morales Zapata

This Jupyter Notebook generates a class to read a automaton's configuration wheter in a file or a manual configuration, and transform it into a regular
expression (REGEX).

We have decided to implement the use of a couple of libraries to help us in the process.

- **os** is to get the directory of the file
- **random** is to be used within the Sipser's Algorithm
- **regex** to a straightforward use on the validation of a word
- **deepcopy** for its use within the process

We would like to make some observations about the following implementation. The methods programmed might not be the most optimized ones, but they give a right result 😉.

Furthermore, they might be some cases (or a bug 🥺) when we call the `convert_automaton()` method and its result is a empty string. Thus, we highly recommend you to run as many times as you want/need for this to be fixed. The latter is due to the `random` package we are using on said method.

Another thing to be considered, is the fact that, for our practicity, we decided to add some auxiliary functions for the main methods of the assignment.

In [1]:
import os
import random
import regex as re
from copy import deepcopy

# Regular Expression Generator Class
---

In [2]:
class  RegularExpressionGenerator():
    '''
    Class to instantiate the properties in the NonDeterminsiticAutomaton object
    '''
    
    def initialize(self, q, sigma, delta, q0, f, empty_symbol = 'NA'):
        '''
        Method that will work only for us to set the NFA's configuration and set its own properties.
        
        PARAMETERS:
        - q: LIST of strings representating the automata's states
        - sigma: LIST of symbols contained in the alphabet 
        - delta: DICTIONARY OF PAIRS which are the tuple of two strings, and a LIST of values q which the automaton's possible
        states
        - q0: STRING representating the initial state
        - empy_symbol: STRING representating the empty transition in the automaton  
        - f: LIST of final states
        
        OUTPUT:
        There is no return for the user on this method more than the automaton setted.
        '''
        # Start by making some changes on the input data so we can work with it in my further methods
        
        # ---- Changing everything to strings -----
        
        # States 'q'
        for i in range(len(q)): 
            q[i] = str(q[i]).replace(' ','')
        
        # Alphabet 'sigma'
        for i in range(len(sigma)):
            sigma[i] = str(sigma[i]).replace(' ','')
        
        # Transition functions 'delta'
        delta_fixed = {}
        for i in delta.keys():
            llave = str(i)
            llave = llave.replace(' ','').replace(',' ,'')
            valor = []
            valor_te = delta[i]
            if type(delta[i]) != list:
                valor.append(str(valor_te).replace(' ','').replace(',',''))
            else: 
                for element in valor_te:
                    valor.append(str(element).replace(' ','').replace(',',''))
            delta_fixed[llave] = valor
        
        # Initial state 'q0'
        q0 = str(q0).replace(' ','')
        
        # Final states 'f'
        for i in range(len(f)):
            f[i] = str(f[i]).replace(' ','')
        
        # Store the values of q
        self.q = q
        # Store the values of sigma
        self.sigma = sigma
        # Store the values of delta
        self.delta = delta_fixed
        # Store the initial state q0
        self.q0 = q0
        # Store the list of final states
        self.f = f
        # Store the string for empty set
        self.empty = empty_symbol
        # Establish the current state of the word (as there is no word, it is q0)
        self.current = self.q0
    
    
    def load_from_file(self, name, wd = os.getcwd()):
        '''
        This function works to load a .TXT file where you have an automata's configuration where each parameter must be 
        separeted from an enter ('\n') and must start with brackets, just as a set.
        The order of the parameters is: q, sigma, delta, q0, f.
        
        PARAMETERS: 
        - name: It MUST be a string type standing for the file's name followed by the .txt termination. e.g 'filename.txt'
        - wd (OPTIONAL): Stands for the Working Directory where your .txt file is located. The default option is the current working
          directory. It MUST be a string type.
        
        OUTPUT:
        This function does not return anything to the user but it establishes the Automata's configuration. This one
        must be seen as an alternative version of the initialize() method with the difference that here you are providing
        the information from an external file. 
        '''
        
        # Esblish the working directory
        wd = wd +'/'+ name
        # Open the file for reading only
        file = open(wd, 'r')
        # Create a list with every line 
        l = file.read().splitlines()
        
        # Test that the user gave a 5-lines input in the .txt file 
        if len(l) != 5:
            print('Your file is not in the right format.\nUse the method txt_help() to have an idea on how it should be')
        
        # ---- Assign the proper values to the Automata ----
        # Creating list-like and elements as empty lists
        self.q = []
        self.sigma = []
        self.delta = {}
        self.f = []
        self.empty = 'NA'
        
        # Enter to the line in the txt where the q values should be and apply the correspondant changes
        values_q = l[0].replace('{','').replace('}','').replace(' ','').split(',')
        for element in values_q: 
            self.q.append(element)
        
        # Assign the values for sigma 
        values_sigma = l[1].replace('{','').replace('}','').replace(' ','').split(',')
        for element in values_sigma:
            self.sigma.append(element)
            
        # Assign the values for delta
        values_delta = l[2].replace('{','').replace('}','').replace(' ','').split(',')
        delta_new = {}
        for element in values_delta:
            value_ls = []
            i = element.rfind('-')
            key = element[:i]
            value = element[i+1:]
            if key not in delta_new.keys():
                value_ls.append(value)
                delta_new[key] = value_ls
            else:
                delta_new[key].append(value)
            # This part is just to take advantage of the code and extract the empty symbol
            if 'epsilon' in element:
                self.empty = 'epsilon'
            if 'lambda' in element: 
                self.empty = 'lambda'            
        self.delta = delta_new
            
        # Assign the values for q0
        self.q0 = l[3].replace(' ','')
        
        # Assign the values for f
        values_f = l[4].replace('{','').replace('}','').replace(' ','').split(',')
        for element in values_f: 
            self.f.append(element)
            
        # To tell where my word is at, remain in the initial state
        self.current = self.q0
        
    def convert_automaton(self):
        '''
        Method called to convert the automaton generated in the configuration of the file or a manualy written, into its equivalent regular expresion.

        PARAMETERS:
        None

        OUTPUT:
        A regular expression in a STRING data type.
        '''
        # Call the auxiliary functions
        resultado = self.convert_automaton_aux()
        while len(resultado) == 0:
            resultado = self.convert_automaton_aux()
        
        # Prepare the result to return it to the user
        respuesta = list(resultado.keys())
        respuesta = respuesta[0]
        index = respuesta.index('-')
        regex = respuesta[index+1:]

        # Clean the output if needed
        if 'lambda+' in regex:
            regex = regex.replace("lambda+",'')
        if '+lambda' in regex:
            regex = regex.replace("+lambda",'')
        if '(lambda+)' in regex:
            regex = regex.replace("(lambda+)",'')
        if '(+lambda)' in regex:
            regex = regex.replace("(+lambda)",'')
        if '(lambda)' in regex:
            regex = regex.replace("(lambda)",'')
        if 'lambda' in regex:
            regex = regex.replace("lambda",'')
        if 'vacio+' in regex:
            regex = regex.replace("vacio+",'')
        if '+vacio' in regex:
            regex = regex.replace("+vacio",'')
        if '(vacio+)' in regex:
            regex = regex.replace("(vacio+)",'')
        if '(+vacio)' in regex:
            regex = regex.replace("(+vacio)",'')
        if '(vacio)' in regex:
            regex = regex.replace("(vacio)",'')
        if 'vacio' in regex:
            regex = regex.replace("vacio",'')
        regex = regex.replace('U','+')
        
        # Return the regular expression
        return regex

    def process_symbol(self, state, symbol):
        '''
        This method receives a string representing the state and the symbol to process and returns 
        the set of possible next states
        
        INPUTS
        - sate: STRING which indicates the state from which you are testing
        - symbol: STRING which indicates the symbol to test along with the state to see the possible outputs
        
        OUPUT
        - Possible next states: LIST of all the possible results by testing the state and symbol in the automata's 
        configuration.
        '''
        # Validate if the states belongs to the automaton
        if state not in self.q:
            raise Exception('Not a valid state')
        
        key_symb = state + '-' + symbol
        key_empt = state + '-' + self.empty

        # Condition to check if in case the state-symbol notation is not in any transition function...
        if key_symb not in self.delta.keys():
            # ... verify if there is any with state-empty transition function
            if key_empt in self.delta.keys():
                # List of the next possible states using an empty transition
                siguientes_lambda = self.empty_transitions(state)
                
                # List of all the possible values to make a comparition after, to dismiss those states that are not defined as a result of a transition function
                delta_values = [item for value in self.delta.values() for item in value]
                
                # Copy of the siguientes_lambda list to iterate in the original values without expecting erros in a modified list
                sig_lambda_copy = siguientes_lambda
                for est in sig_lambda_copy:
                    # Specifics conditions where there might be a problem or error based on the results of the 
                    if est not in delta_values:
                        siguientes_lambda.remove(est)
                    elif est + '-' + self.empty not in self.delta.keys():
                        siguientes_lambda.remove(est)
                    elif self.delta[est + '-' + self.empty] != est:
                        siguientes_lambda.remove(est)
                
                # List with the all posible states defined in the transition functions
                siguientes = []
                for element in siguientes_lambda:
                    key_symb_sig = element + '-' + symbol
                    if key_symb_sig not in self.delta.keys():
                        continue
                    else:
                        siguientes += self.delta[key_symb_sig]
                if len(siguientes) == 0:
                    return print('This is not defined in your transition function, so you will remain in', state,
                                 '\n\nIf you think this is a mistake, please check your input and your configuration :)')
                
                # List to store the states with no repititions
                fixed = []
                for estado in siguientes:
                    if estado not in fixed: 
                        fixed.append(estado)
                #print('\nThese are your possible final states after that configuration:\n',fixed)
                
                # Return the final list
                return fixed
            
            # If there is no transition function posible, the processsymbol cannot be executed
            return print('This is not defined in your transition function, so you will remain in', state,
                         '\n\nIf you think this is a mistake, please check your input and your configuration :)')
        
        else:
            # Do the same process in case there indeed is any transition function with the state-symbol
            siguientes_lambda = []
            if key_empt in self.delta.keys():
                siguientes_lambda += self.empty_transitions(state)
            
            siguientes = self.delta[key_symb]
            for element in siguientes: 
                if element+'-'+self.empty in self.delta.keys():
                    siguientes_lambda += self.empty_transitions(element)
            
            delta_values = [item for value in self.delta.values() for item in value]
            
            sig_lambda_copy = siguientes_lambda
            for est in sig_lambda_copy:
                if est not in delta_values:
                    siguientes_lambda.remove(est)
                elif est + '-' + self.empty not in self.delta.keys():
                    siguientes_lambda.remove(est)
                elif est not in self.delta[est + '-' + self.empty]:
                    siguientes_lambda.remove(est)
                    
            siguientes = siguientes + siguientes_lambda
            fixed = []
            for estado in siguientes:
                if estado not in fixed: 
                    fixed.append(estado)
            #print('\nThese are your possible final states after that configuration:\n',fixed)
            return fixed

    def empty_transitions(self, state):
        '''
        This method will be used to identify the possible next states when using an empty transition
        from the state you want. This is the method we expect the user to use in their automaton.
        
        PARAMETERS:
        - state: a STRING representing the states in which the user wants to get the set of possible states reachable using empty transitions.
        
        OUTPUT:
        - result: cleaned LIST of all posibles states to be reached from the state given in the parameters
        '''
        # Calling the complementary recursive function
        lista = self.empty_transitionss(state)
        
        # List to stored the result cleaned
        result =  []
        # Condition to check if the user has incorrectly input the parameters
        if str(type(lista)) == "<class 'NoneType'>":
            print('You cannot use',self.empty,'with this state. You remain in',state)
            return list(state)
        # Iteration in the result of the complementary function result to store the cleaned data into result
        for element in lista:
            if element not in result:
                result.append(element)
        #print('You can move to the states in the list with lambda: ' +str(result))
        return result
    
    def check_word(self, word):
        '''
        This method receives a string and use the regular expression library
        `regex` to compile the equivalent regular expression and return all matches
        of the expression in the string.

        PARAMETERS:
        - word: a STRING type of the word to be matched with the regular expression

        OUTPUT:
        A list of possible matches of the word within any equivalent regular expression of the automaton
        '''
        # # Call the method to generate the regural expression
        reg_ex = self.convert_automaton()
        while len(reg_ex) == 0:
            reg_ex = self.convert_automaton()

        # enerate the matches with the `regex` package
        matches = [set(i) for i in list(re.findall(reg_ex, word))]
        
        print(f'Regular Expression:\n{reg_ex}\n\nMatches of {word}:\n{matches}')

        return matches

# ---------------------------- AUXILIAR FUNCTIONS ---------------------------------------------        
    
    def convert_to_g(self):
        
        '''
        This function adds the initial and final transactions to convert any automaton
        (DFA or NFA) into a GNFA.
        '''
        # Self copy with deepcopy
        gnfa = deepcopy(self)
        # Add a empty symbol if a DFA was ingressed
        if gnfa.empty == 'NA':
            gnfa.empty = 'lambda'
        # Add the Initial and Last One state to the old initial and final states
        anadir = 'Initial'+'-'+ gnfa.empty
        gnfa.delta[anadir] = [gnfa.q0]
        gnfa.q0 = 'Initial'
        
        finales_old = gnfa.f
        for element in finales_old:
            anadir_temp = element+'-'+gnfa.empty
            gnfa.delta[anadir_temp] = ['Lastone']
        gnfa.f = ['Lastone']
        
        gnfa.q.append('Initial')
        gnfa.q.append('Lastone')
        # Return the GNFA with a single initial and final state
        return gnfa
    
    def obtain_transition_letters(self, q,delta, q_1, q_2):
        '''
        Auxiliar function to get the transition letters.
        '''
        transiciones = []
        respuesta = ''
        # Verify which keys-values are valid
        for element in list(delta.keys()):
            if (q_1 in element) and (q_2 in delta[element]):
                # Append to the transition list the letter
                corte = element.index('-')
                anadir = element[corte+1:]
                transiciones.append(anadir)
        # Verify if there were more than a letter connecting such states
        if len(transiciones) == 0:
            respuesta = 'vacio'
        elif len(transiciones) == 1:
            respuesta = transiciones[0]
        elif len(transiciones) > 1:
            respuesta = '|'.join(transiciones)

        # Return the answer
        return respuesta
    

    def corregir_palabra(self, r1, r2, r3, r4, gnfa):
        '''
        Auxiliar function to fix the regular expression based on its parts.
        '''
        # Check for R1 to obtain R1R2*
        if r1 == gnfa.empty:
            if (r2 == 'vacio') or (r2 == gnfa.empty):
                r1r2 = ''
            else:
                r1r2 = f'({r2})*'
        else:
            if (r2 == 'vacio') or (r2 == gnfa.empty):
                r1r2 = f'({r1})'
            else:
                r1r2 = f'({r1})({r2})*'

        # Check for R3 to obtain R1R2*R3
        if r3 == 'vacio':
            r1r2r3 = ''
        elif r3 == gnfa.empty:
            if r1r2 == '':
                r1r2r3 = ''
            else:
                r1r2r3 = f'{r1r2}'
        else:
            r1r2r3 = f'{r1r2}({r3})'
        
        # Check for R4 to obtain R1R2*R3UR4
        if (r4 == 'vacio') or (r4 == gnfa.empty):
            r1r2r3r4 = r1r2r3
        else:
            r1r2r3r4 = f'{r1r2r3}|({r4})'
        
        # Return the regular expression for R1R2*R3UR4
        return r1r2r3r4
    
    def convert_automaton_aux(self):
        '''
        Auxiliar function to convert the automaton into a regular expression.
        '''
        gnfa = self.convert_to_g()
        q = gnfa.q
        initial = gnfa.q0
        lastone = gnfa.f
        delta = gnfa.delta
        k = len(gnfa.q)
        while k > 2:
            # Random q_rip
            q_rip = random.choice(q)
            # Veryfy that it is not the Initial nor LastOne states
            while (q_rip in lastone) or (q_rip == initial):
                q_rip = random.choice(q)

            # -------------------------INPUTS OF Q_RIP---------------------------------- 
            # List for the values/states that can reach q_rip
            values = list(delta.values())
            keys = list(delta.keys())
            q_i_lista = []
            for i in range(len(list(delta.keys()))):
              if str(type(values[i])) == "<class 'NoneType'>":
                break
              if q_rip in values[i]:
                corte = keys[i].index('-')
                anadir = keys[i][:corte]
                if anadir not in q_i_lista:
                  q_i_lista.append(anadir)
                
            # --------------------------OUTPUTS OF Q_RIP---------------------------------
            # List for the values/states that can be reached from q_rip
            q_j_lista = []
            for element in list(delta.keys()):
                if (q_rip in element):
                    if str(type(delta[element])) == "<class 'NoneType'>":
                        break
                    for p in delta[element]:
                        if p not in q_j_lista:
                            q_j_lista.append(p)
            
            # --------------------------OMIT ANY Q_RIP RELATION/TRANSITION------------------
            for q_i in q_i_lista:
                for q_j in q_j_lista:
                    if (q_i not in lastone) and (q_j != initial): 
                        r1 = self.obtain_transition_letters(q,delta, q_i, q_rip)
                        r2 = self.obtain_transition_letters(q,delta, q_rip, q_rip)
                        r3 = self.obtain_transition_letters(q,delta, q_rip, q_j)
                        r4 = self.obtain_transition_letters(q,delta, q_i, q_j)
                        
                        transicion = self.corregir_palabra(r1, r2, r3, r4, gnfa)
                        if len(transicion)==0:
                          transicion_vacia = q_i + '-' + gnfa.empty
                          delta[transicion_vacia] = [q_j]
                        else:
                          transicion_1 = q_i+'-'+transicion
                          delta[transicion_1] = [q_j]
            
            # Remove q_rip from all the states q
            q.remove(q_rip)
            
            # Remoce all its outputs
            copia_delta = delta
            for key in list(copia_delta.keys()):
                if q_rip in key:
                    del delta[key]
            
            # Remove all its inputs
            for key in list(copia_delta.keys()):
                if str(type(delta[key])) == "<class 'NoneType'>":
                    break
                if q_rip in delta[key]:
                    eliminar_qrip = delta[key]
                    if len(eliminar_qrip) == 1:
                        del delta[key]
                    else:
                      nueva_lista = eliminar_qrip.remove(q_rip)
                      delta[key] = nueva_lista
            # Remove all qi'relation with q_rip
            k = len(q)
        return delta

    def empty_transitionss(self, state, contador = 0):
        '''
        Auxilat function that will be used to identify the possible next states when using an empty transition
        from the state you want. However, this method is used in its recursive form only, and as a tool to
        ingress its output to the method/function we expect the user to use in their automaton.
        '''
        test = state+'-'+self.empty
        
        # Base condition where the state-emty notation is not included in any of the transition functions
        if test not in self.delta.keys():
            return
        
        # List of all posible states based on the transition functions
        posibles_temp = self.delta[test]
        
        # List to store all the states to be accepted as an output
        aceptados = []
        aceptados.append(state)
        
        # Modification to the counter
        contador += 1
        # If we are analizing more states than the ones we have, of course we are in a cycle 
        if len(self.q)*2 < contador:
            del contador
            contador = 0 
            return
        # Iteration wit the recursivity when we check all the posible states already stored
        for stat in posibles_temp:
            aceptados.append(stat)
            # Resursion
            f = self.empty_transitionss(stat, contador)
            # Condition to check if the recursion has finished yet
            if f != None:
                # Loop to append each of the elements of the recursivity's output
                for element in f:
                    aceptados.append(element)            
        return aceptados
        

# Tests
---
## File Test

In [4]:
# Initialize Class
a = RegularExpressionGenerator()

# Load the automaton configuration from file
a.load_from_file('prueba.txt', wd = r'UPY/Autómatas 06') # Make sure to use your own path!

In [5]:
# Convert Automaton method
a.convert_automaton() # Run as needed (sorry for the bug ☹)

''

In [6]:
# Verify the transition function of the automaton
a.delta

{'q0-0': ['q3'],
 'q0-1': ['q1'],
 'q1-0': ['q2', 'q3'],
 'q1-1': ['q1'],
 'q2-0': ['q2'],
 'q2-1': ['q1'],
 'q3-0': ['q3'],
 'q3-1': ['q3']}

In [7]:
# Check word method
a.check_word('110')

Regular Expression:
(((1)(1)*(0)|((1)(1)*(0))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))*(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)))*(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))*(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)))|((1)(1)*(0)))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))*(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)))*(|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))*(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))*(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0))|(0|(1)(1|(1)(1)*(1)|(1))*(0)|(0)))*))

Matches of 110:
[{'', '110', '0', '1'}]


[{'', '0', '1', '110'}]

## Manual Test

In [8]:
# Initialize Class
p = RegularExpressionGenerator()

In [9]:
# Manually enter the configuration
q = ['W','X','Y','Z']
sigma = ['a','b']
delta = {'W-a':'X','W-b':'X','X-a':'Z','X-b':'Y','Y-a':'Y','Y-b':'Y','Z-a':'Z','Z-b':'Z'}
q0 = 'W'
f = ['Y']

# Initialize method with the precious configuration
p.initialize(q=q, sigma=sigma, delta=delta, q0=q0, f=f)

In [10]:
# Convert Automaton method
p.convert_automaton() # Run as needed (sorry for the bug ☹)

'((a|b))((b|(b)(a|b)*(a|b)|(b))(a|b)*)'

In [11]:
# Verify the transition function of the automaton
p.delta

{'W-a': ['X'],
 'W-b': ['X'],
 'X-a': ['Z'],
 'X-b': ['Y'],
 'Y-a': ['Y'],
 'Y-b': ['Y'],
 'Z-a': ['Z'],
 'Z-b': ['Z']}

In [12]:
# Check word method
p.check_word('aaaaba')

Regular Expression:
((a|b)((b|(b)(a|b)*(a|b)|(b))(a|b)*))

Matches of aaaaba:
[{'', 'ba', 'a', 'aba', 'b'}]


[{'', 'a', 'aba', 'b', 'ba'}]